In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
img_id = "CA184632V0342"

In [11]:
import src.load.load_image as ld

#path_data_fld = "C:/Users/Felix/Documents/GitHub/Antarctic_TMA/data"
path_data_fld = "/data_1/ATM/data_1/aerial/TMA/downloaded"

# load image to geo-reference
image = ld.load_image(path_data_fld + f"/{img_id}.tif")

print(image.shape)

(9522, 10140)


In [12]:
import src.base.connect_to_database as ctd

sql_string = f"SELECT * FROM images WHERE image_id='{img_id}'"
conn = ctd.establish_connection()
data = ctd.execute_sql(sql_string, conn)

azimuth = data.iloc[0]['azimuth']
month = data.iloc[0]['date_month']

azimuth = 360 - azimuth + 90

print(azimuth, month)

sql_string_approx = f"SELECT ST_AsText(footprint_approx) AS footprint_approx FROM images_extracted WHERE image_id='{img_id}'"
data_approx = ctd.execute_sql(sql_string_approx, conn)
footprint_approx = data_approx.iloc[0]['footprint_approx']
print(footprint_approx)

338.8565 12
POLYGON((-1916347.9352607953 745784.3927205991,-1919432.997279401 737807.3052607954,-1927410.0847392047 740892.3672794009,-1924325.022720599 748869.4547392046,-1916347.9352607953 745784.3927205991))


In [13]:
import numpy as np
mask = np.ones_like(image)
pad_width = 500

mask[:pad_width, :] = 0  # Top edge
mask[-pad_width:, :] = 0  # Bottom edge
mask[:, :pad_width] = 0  # Left edge
mask[:, -pad_width:] = 0  # Right edge

In [14]:
import src.georef.georef_sat as gs

georefSat = gs.GeorefSatellite(min_tps_final=25,
                               enhance_image=False, locate_image=True, tweak_image=True,filter_outliers=True)

transform, residuals, tps, conf = georefSat.georeference(image, footprint_approx, mask, azimuth, month)


Geo-reference image by satellite
Adjusted image resolution with zoom-factor (0.0883, 0.0859)
Adjusted image resolution with zoom-factor (0.0883, 0.0859)
Locate image position
  Check tile 1 (Coords: [-7380, -7386], Order 1)
  0 points were found in this tile.
  Check tile 2 (Coords: [-7380, 0], Order 1)
  0 points were found in this tile.
  Check tile 3 (Coords: [-7380, 7386], Order 1)
  206 points were found in this tile.
  Check tile 4 (Coords: [0, -7386], Order 1)
  0 points were found in this tile.
  Check tile 5 (Coords: [0, 7386], Order 1)
  93 points were found in this tile.
  Check tile 6 (Coords: [7380, -7386], Order 1)
  0 points were found in this tile.
  Check tile 7 (Coords: [7380, 0], Order 1)
  0 points were found in this tile.
  Check tile 8 (Coords: [7380, 7386], Order 1)
  2 points were found in this tile.
  Check tile 9 (Coords: [-14760, -14772], Order 2)
  6 points were found in this tile.
  Check tile 10 (Coords: [-14760, -7386], Order 2)
  4 points were found in t

In [15]:
import src.georef.snippets.apply_transform as af

output_fld = "/data_1/ATM/data_1/georef/sat"

af.apply_transform(image, transform, f"{output_fld}/{img_id}.tif")

np.savetxt(f"{output_fld}/{img_id}_transform.txt", transform)
np.savetxt(f"{output_fld}/{img_id}_points.txt", tps)